## Importa funções para o pré-processamento

In [1]:
from pre_processamento import *

from dlisio import dlis
import pandas as pd
import numpy as np

# A função 'glob' do módulo 'glob' é usada para procurar todos os arquivos em um diretório com determinada extensão
import glob

## Carrega os dados

In [2]:
nomes_arquivos = []     # Armazena os nomes dos arquivos .dlis
leituras_dlis = []      # Armazena as leituras dos arquivos .dlis
nomes_anp = []          # Armazena os nomes obtidos das leituras

for file in glob.glob(r'**/Data' + "/*.dlis", recursive=True):
    # Salva o nome do arquivo
    nomes_arquivos.append(file)

    # Salva os dados da leitura
    leitura, *tail = dlis.load(f'{file}')
    leituras_dlis.append(leitura)

    # Salva o nome do poço
    nome = leitura.origins[0].well_name
    nomes_anp.append(nome)

In [3]:
print(nomes_arquivos)
print(nomes_anp)

['Data\\1-brsa-551-se_brsa_raw.dlis', 'Data\\1-brsa-574-se_brsa_raw.dlis', 'Data\\1-brsa-595-se_brsa_raw-1.dlis', 'Data\\1-brsa-605-se_brsa_raw.dlis', 'Data\\1-brsa-659-se_brsa_raw.dlis', 'Data\\3-CP-1847-SE.dlis', 'Data\\3-CP-1848-SE.dlis', 'Data\\3-CP-1851-SE.dlis', 'Data\\3-CP-1853-SE.dlis', 'Data\\3-CP-1855-SE.dlis', 'Data\\3-CP-1857-SE.dlis']
['1-FSG-1-SE', '1-FSJQ-1-SE', '1-BRSA-595-SE', '1-BRSA-605-SE', '1-BRSA-659-SE', '3-BRSA-900-SE', '3-BRSA-889-SE', '3-BRSA-912-SE', '3-BRSA-897-SE', '3-BRSA-910-SE', '3-BRSA-907-SE']


## Padroniza nomes fora do padão 'X-BRSA-XXX-SE'

In [4]:
nomes_anp[0] = '1-BRSA-551-SE'
nomes_anp[1] = '1-BRSA-574-SE'

In [5]:
nomes_anp

['1-BRSA-551-SE',
 '1-BRSA-574-SE',
 '1-BRSA-595-SE',
 '1-BRSA-605-SE',
 '1-BRSA-659-SE',
 '3-BRSA-900-SE',
 '3-BRSA-889-SE',
 '3-BRSA-912-SE',
 '3-BRSA-897-SE',
 '3-BRSA-910-SE',
 '3-BRSA-907-SE']

## Cria dicionário para armazenar os dados e respectivos nomes

In [6]:
# Casa itens da lista 'nome_anp_abreviados' com os itens da lista 'leituras_dlis'
pares = zip(nomes_anp, leituras_dlis)

# Cria dicionário 'dli_dict'
dli_dict = dict(pares)
dli_dict

{'1-BRSA-551-SE': LogicalFile(AIT_SONIC_TLD_MCFL_018PUP),
 '1-BRSA-574-SE': LogicalFile(GEOLOAD.1),
 '1-BRSA-595-SE': LogicalFile(GEOLOAD.1),
 '1-BRSA-605-SE': LogicalFile(AIT_SONIC_TLD_MCFL_048PUC),
 '1-BRSA-659-SE': LogicalFile(GEOLOAD.1),
 '3-BRSA-900-SE': LogicalFile(GEOLOAD.1),
 '3-BRSA-889-SE': LogicalFile(GEOLOAD.1),
 '3-BRSA-912-SE': LogicalFile(GEOLOAD.1),
 '3-BRSA-897-SE': LogicalFile(GEOLOAD.1),
 '3-BRSA-910-SE': LogicalFile(GEOLOAD.1),
 '3-BRSA-907-SE': LogicalFile(GEOLOAD.1)}

## Separa TODAS as curvas presentes nos .dlis de cada poço

In [7]:
channels_dict = {}

for key, poco in dli_dict.items():
    channels_list = []
    for frame in poco.frames:
        channels = frame.channels
        channels_list.append([channel.name for channel in channels])
    channels_dict[key] = sum(channels_list, [])

## Concatena os arquivos CSV das curvas em um único arquivo

In [10]:
import os

pasta = "Curvas_CSV"

# Lista para armazenar os dataframes repectivos aos .csv
df_list = []

# Itera sobre a pasta Curvas_CSV
for arquivo in os.listdir(pasta):
    file_path = os.path.join(pasta, arquivo)

    # Lê o arquivo CSV como um DF
    df = pd.read_csv(file_path)
    df_list.append(df)

# Concatena os DFs ao longo do eixo das colunas
df_concat = pd.concat(df_list, axis=0, ignore_index=True)

# Salva o DF concatenado em um .csv
df_concat.to_csv("curvas_pocos.csv", index=False)

print("Arquivo curvas_pocos.csv criado com sucesso")

Arquivo curvas_pocos.csv criado com sucesso


## Analisa os 'frames' dos poço 1-BRSA-605-SE

#### Quantidade de 'frames' no poço 1-BRSA-605-SE

In [52]:
dli_dict['1-BRSA-605-SE'].frames

[Frame(60B), Frame(10B), Frame(30B), Frame(20B)]

#### Cria um 'dataframe' para cada 'frame'

In [53]:
dataframes_605 = {}
poco = dli_dict['1-BRSA-605-SE']

for frame in poco.frames:
    indice = poco.frames.index(frame)

    curves = pd.DataFrame(frame.curves())

    dataframes_605[indice] = curves

#### Salva os dataframes como arquivos CSV

In [56]:
for key, value in dataframes_605.items():
    file_name = f"Frames_605/frame_{key}.csv"
    value.to_csv(file_name, index=False)
    print(f"Arquivo {file_name} criado com sucesso.")

Arquivo Frames_605/frame_0.csv criado com sucesso.
Arquivo Frames_605/frame_1.csv criado com sucesso.
Arquivo Frames_605/frame_2.csv criado com sucesso.
Arquivo Frames_605/frame_3.csv criado com sucesso.


In [ ]:
help(dli_dict['1-BRSA-605-SE'].frames[0])

## Cria dataframes para os poços

Alguns poços possuem mais de um 'frame'.

Irei criar um 'dataframe' para cada um desses 'frames'.

Irei fazer um 'JOIN' usando a coluna 'TDEP'.

### Teste 1

#### Cria um dataframe para cada frame do poço

In [34]:
curvas_escolhidas = ['TDEP', 'GR', 'NPHI', 'RHOB', 'RHOZ', 'DRHO', 'BSZ', 'BS', 'CALI', 'DCALI', 'DCAL', 'PE', 'DTC', 'ILD', 'RILD', 'IEL', 'AIT90', 'AHT90', 'RT90', 'AT90', 'AO90', 'RT', 'AF90', 'AHF90', 'AFH90', 'LLD', 'RLLD', 'HDRS', 'HLLD', 'LL7', 'RLL7']

# Iniciando o teste com um poço apenas
poco = dli_dict['1-BRSA-605-SE']

# Dicionário para armazenar os dataframes dos frames do poço 605
dataframes_605 = {}

# Iterando sobre os frames do poço
for frame in poco.frames:
    # indice do frame
    indice = poco.frames.index(frame)
    
    # obtendo os nomes das curvas 
    channels = frame.channels
    curvas_nomes = [channel.name for channel in channels
                    if channel.name in curvas_escolhidas]
 
    # obtendo os valores das curvas
    curvas_valores = frame.curves()

    dataframes_605[indice] = pd.DataFrame(curvas_valores, columns=curvas_nomes)


In [35]:
df_merged = dataframes_605[0]

for i in range(1, len(dataframes_605)):
    df_merged = pd.merge(df_merged, dataframes_605[i], on='TDEP', how='outer')

In [46]:
df_sorted = df_merged.sort_values(by='TDEP')
df_sorted.to_csv("teste_605.csv", index="False")

In [66]:
dlis_df_dict = {}   # Conterá os dataframes respectivos aos poços

# Curvas de perfis escolhidas
curvas_escolhidas = ['TDEP', 'GR', 'NPHI', 'RHOB', 'DRHO', 'HDRS', 'LLD', 'BSZ', 'BS', 'CALI', 'DCALI', 'DCAL', 'PE', 'DTC']

# Iterando sobre os arquivos lógicos de todos os poços (que estão armazenados em 'dli_dict'),
# '.values()' se refere aos valores do dicionário (não às chaves)
for chave, poco in dli_dict.items():

    # Armazenando as curvas que serão utilizadas em uma lista
    curvas_utilizadas = [
        channel.name                                    # Os elementos da lista serão os nomes das curvas
        for channel in poco.channels                    # As curvas são acessadas por meio de 'poco.channels'
        if channel.name in curvas_escolhidas            # As curvas que não utilizaremos não serão armazenadas na lista
    ]
    conjunto_aux = set(curvas_utilizadas)
    curvas_utilizadas_sem_duplicados = list(conjunto_aux)
    
    #frames = [np.atleast_1d(frame) for frame in poco.frames]
    #curvas = np.concatenate(frames, axis=0)
    dataframe = pd.DataFrame()
    
    for frame in poco.frames:

        curvas = frame.curves()

        dataframe = pd.concat([dataframe, pd.DataFrame(curvas[curvas_utilizadas_sem_duplicados])])

    # Criando um pandas dataframe do poço respectivo à atual iteração e armazenando o mesmo em dlis_df
    dlis_df_dict[chave] = dataframe


KeyError: 'GR'

In [64]:
dlis_df_dict.keys()

dict_keys(['551', '574', '595', '605', '659', '900', '889', '912', '897', '910', '907'])

## Transforma os valores -999.25 em nulos

In [ ]:
for poco in dlis_df_dict.values():
    poco.replace([-999.25], [None], inplace = True)

## Aplicando os mnemônicos

In [ ]:
aplica_mnemonico(dlis_df_dict, ['BS', 'BSZ'], 'BS')
aplica_mnemonico(dlis_df_dict, ['LLD', 'HDRS'], 'RESD')
aplica_mnemonico(dlis_df_dict, ['RHOB', 'RHOZ'], 'RHOB')

## Renomeia CALI para CAL

In [ ]:
for poco in dlis_df_dict.values():
    renomeia_coluna(poco, 'CALI', 'CAL')

## Adiciona coluna DCAL

In [ ]:
add_DCAL(dlis_df_dict)

## Preenche os poços com curvas faltando

In [ ]:
# Se um dos poços não tiver uma dessas curvas, adicionamos a coluna da curva e mantemos os valores como None
curvas_obrigatorias = ['BS', 'CAL', 'DCAL', 'DRHO', 'DTC', 'GR', 'NPHI', 'PE', 'RESD', 'RHOB', 'TDEP']

# Percorre todos os poços
for poco in dlis_df_dict.values():
    # Percorre todas as curvas obrigatórias
    for curva in curvas_obrigatorias:
        # Se o poço não tiver a curva
        if curva not in poco.columns:
            # Adiciona a coluna e os valores dela = None
            poco[curva] = None

## Temos as seguintes curvas 

In [ ]:
for key, poco in dlis_df_dict.items():
    curvas = sorted(poco.keys())
    print(f"{key}: {curvas}")

551: ['BS', 'CAL', 'DCAL', 'DRHO', 'DTC', 'GR', 'NPHI', 'PE', 'RESD', 'RHOB', 'TDEP']
574: ['BS', 'CAL', 'DCAL', 'DRHO', 'DTC', 'GR', 'NPHI', 'PE', 'RESD', 'RHOB', 'TDEP']
595: ['BS', 'CAL', 'DCAL', 'DRHO', 'DTC', 'GR', 'NPHI', 'PE', 'RESD', 'RHOB', 'TDEP']
605: ['BS', 'CAL', 'DCAL', 'DRHO', 'DTC', 'GR', 'NPHI', 'PE', 'RESD', 'RHOB', 'TDEP']
659: ['BS', 'CAL', 'DCAL', 'DRHO', 'DTC', 'GR', 'NPHI', 'PE', 'RESD', 'RHOB', 'TDEP']
900: ['BS', 'CAL', 'DCAL', 'DRHO', 'DTC', 'GR', 'NPHI', 'PE', 'RESD', 'RHOB', 'TDEP']
889: ['BS', 'CAL', 'DCAL', 'DRHO', 'DTC', 'GR', 'NPHI', 'PE', 'RESD', 'RHOB', 'TDEP']
912: ['BS', 'CAL', 'DCAL', 'DRHO', 'DTC', 'GR', 'NPHI', 'PE', 'RESD', 'RHOB', 'TDEP']
897: ['BS', 'CAL', 'DCAL', 'DRHO', 'DTC', 'GR', 'NPHI', 'PE', 'RESD', 'RHOB', 'TDEP']
910: ['BS', 'CAL', 'DCAL', 'DRHO', 'DTC', 'GR', 'NPHI', 'PE', 'RESD', 'RHOB', 'TDEP']
907: ['BS', 'CAL', 'DCAL', 'DRHO', 'DTC', 'GR', 'NPHI', 'PE', 'RESD', 'RHOB', 'TDEP']


## Remove valores DRHO e DCAL indesejados (Só depois)

In [ ]:
#limita_curva(dlis_df_dict, "DRHO", -0.15, 0.15)
#limita_curva(dlis_df_dict, "DCAL", -2, 2)

## Inverte a ordem das linhas dos dataframes

In [ ]:
for key in dlis_df_dict.keys():
    dlis_df_dict[key] = dlis_df_dict[key].iloc[::-1]

## Ordena as colunas dos dataframes

In [ ]:
ordem_desejada = ['TDEP', 'GR', 'RESD', 'BS', 'CAL', 'DCAL', 'NPHI', 'PE', 'DRHO', 'RHOB', 'DTC']

for key in dlis_df_dict.keys():
    dlis_df_dict[key] = dlis_df_dict[key].reindex(columns=ordem_desejada)

## Salva os dados dos dataframes em arquivos CSV

In [ ]:
for key, value in dlis_df_dict.items():
    file_name = f"Pocos_CSV/poco_{key}.csv"
    value.to_csv(file_name, index=False)
    print(f"Arquivo {file_name} criado com sucesso.")

Arquivo Pocos_CSV/poco_551.csv criado com sucesso.
Arquivo Pocos_CSV/poco_574.csv criado com sucesso.
Arquivo Pocos_CSV/poco_595.csv criado com sucesso.
Arquivo Pocos_CSV/poco_605.csv criado com sucesso.
Arquivo Pocos_CSV/poco_659.csv criado com sucesso.
Arquivo Pocos_CSV/poco_900.csv criado com sucesso.
Arquivo Pocos_CSV/poco_889.csv criado com sucesso.
Arquivo Pocos_CSV/poco_912.csv criado com sucesso.
Arquivo Pocos_CSV/poco_897.csv criado com sucesso.
Arquivo Pocos_CSV/poco_910.csv criado com sucesso.
Arquivo Pocos_CSV/poco_907.csv criado com sucesso.
